# Doing cool stuff with satellite data!
What am I doing?

## Checklist of steps

- [x] Access Google Earth Engine API
- [ ] Download LandSat for growing season in Snake River basin

## Resources for datasets and usage
Google: Earth Engine Developers docs (http://developers.google.com/earth-engine)

NASA: Landsat 8 Bands (https://landsat.gsfc.nasa.gov/landsat-8/landsat-8-bands)

USGS: What are the best Landsat spectral bands for use in my research? (https://www.usgs.gov/faqs/what-are-best-landsat-spectral-bands-use-my-research?qt-news_science_products=0#qt-news_science_products)

USGS: Common Landsat Band RGB Composites (https://www.usgs.gov/media/images/common-landsat-band-rgb-composites)

NASA: How to Interpret Common False Color Images (https://earthobservatory.nasa.gov/features/FalseColor/page6.php)

USGS: Spectral Characteristics Viewer (https://landsat.usgs.gov/spectral-characteristics-viewer)

What we built in the class: Karin’s “Landsat 8 Analyzer” code snippet (https://code.earthengine.google.com/ad2cf9683d62290dcbe8d3b00babe45f)

## Setup

### Install Google Earth Engine API

Must start by following the instructions here: https://developers.google.com/earth-engine/guides/python_install-conda#mac_5

Once that is all good to go you should make sure your `ee` conda environment is activated (you should see `(ee)` at the beginning of your command prompt in the terminal where your username usually is. Get there by running `conda activate ee` (after creating the conda env according to the instructions above).

Make sure you are always in your `ee` conda environment for development, and install all library dependencies only after activating your environment.

To share the environment with other users for reproducibility, run 
`conda env export > $HOME/ee-shared-env.yml`, then have the other person download that file and run `conda env create -f path-to-ee-shared-env.yml`

Once that is all set up and you are in your conda env you can simply import earthengine (`ee`)

### Import and test GEE

In [ ]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Print metadata for a DEM dataset.
print(ee.Image('USGS/SRTMGL1_003').getInfo())

### Success!!!

## Testing an example use case
## NOTE: Not sure if this is useful AT ALL
Source: https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_asset_from_cloud_geotiff.ipynb

In [ ]:
from ee import oauth
from google_auth_oauthlib.flow import Flow
import json

# Build the `client_secrets.json` file by borrowing the
# Earth Engine python authenticator.
client_secrets = {
    'web': {
        'client_id': oauth.CLIENT_ID,
        'client_secret': oauth.CLIENT_SECRET,
        'redirect_uris': [oauth.REDIRECT_URI],
        'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
        'token_uri': 'https://accounts.google.com/o/oauth2/token'
    }
}

# Write to a json file.
client_secrets_file = 'client_secrets.json'
with open(client_secrets_file, 'w') as f:
  json.dump(client_secrets, f, indent=2)

# Start the flow using the client_secrets.json file.
flow = Flow.from_client_secrets_file(client_secrets_file,
    scopes=oauth.SCOPES,
    redirect_uri=oauth.REDIRECT_URI)

# Get the authorization URL from the flow.
auth_url, _ = flow.authorization_url(prompt='consent')

# Print instructions to go to the authorization URL.
oauth._display_auth_instructions_with_print(auth_url)
print('\n')

# The user will get an authorization code.
# This code is used to get the access token.
code = input('Enter the authorization code: \n')
flow.fetch_token(code=code)

# Get an authorized session from the flow.
session = flow.authorized_session()

## Generating datasets/assets in GEE then exporting them for use here
Useful info: https://developers.google.com/earth-engine/guides/exporting#configuration-parameters

My understanding is that the best way to do this is to use the JavaScript API in GEE to access and process datasets, which can then be exported and loaded here for use in models, etc.

Still figuring this out though.

### Loading an existing COG (Cloud Optimized GeoTiff)
I thiiiiiink then you can access the asset using something like the following (which uses an example COG, or Cloud Optimized GeoTiff). The request body is an instance of an EarthEngineAsset.

This example COG was generated using this example script: https://code.earthengine.google.com/?scriptPath=Examples%3ACloud%20Masking%2FSentinel2

In [ ]:
# Request body as a dictionary.
request = {
  'type': 'IMAGE',
  'gcs_location': {
    'uris': ['gs://ee-docs-demos/COG_demo.tif']
  },
  'properties': {
    'source': 'https://code.earthengine.google.com/d541cf8b268b2f9d8f834c255698201d'
  },
  'startTime': '2016-01-01T00:00:00.000000000Z',
  'endTime': '2016-12-31T15:01:23.000000000Z',
}

from pprint import pprint
pprint(json.dumps(request))

### Send the request
Make the POST request to the Earth Engine CreateAsset endpoint.

This will place the asset in your assets folder of your GEE account.

In [ ]:
# Send the request
# Where Earth Engine assets are kept.
project_folder = 'earthengine-legacy'
# Your user folder name and new asset name.
asset_id = 'users/elliothohn/sample_asset'

url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'

response = session.post(
  url = url.format(project_folder, asset_id),
  data = json.dumps(request)
)

pprint(json.loads(response.content))

## Example 2 - uploading a set of image tiles into a single asset using a manifest file

Source: https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Uploading_image_tiles_as_a_single_asset_using_a_manifest.ipynb

#### NOTE: this is not working right now, and I don't think I actually need it quite yet (because I think I can start by using assets within GEE without having to store anything in Google Cloud Storage yet). Once I'm at that stage (assuming that's actually necessary/useful?) maybe move this into it's own notebook.

In [ ]:
# URLs of a few tiles.
urls = [
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_36_-120.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_36_-122.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_36_-124.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_38_-120.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_38_-122.tif',
  'http://data.ess.tsinghua.edu.cn/data/fromglc10_2017v01/fromglc10v01_38_-124.tif'
]

# You need to have write access to this bucket.
bucket = 'imagery-testing'

# Pipe curl output to gsutil.
for f in urls:
  filepath = bucket + '/' + f.split('/')[-1]
  !curl {f} | gsutil cp - {filepath}

## TensorFlow example
Source: https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_TensorFlow_AI_Platform.ipynb


In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import tensorflow as tf
print(tf.__version__)

### Creating training data

Read this: https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb